In [2]:
import pandas as pd
import numpy as np

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/human-activity-recognition-with-smartphones")

print("Path to dataset files:", path)

/Users/ludoviclepic/Documents/MINES/3A/MVA/Time series/Code/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 24.5M/24.5M [00:02<00:00, 11.9MB/s]

Extracting files...


Path to dataset files: /Users/ludoviclepic/.cache/kagglehub/datasets/uciml/human-activity-recognition-with-smartphones/versions/2


In [18]:
## load the data
data = pd.read_csv('/Users/ludoviclepic/.cache/kagglehub/datasets/uciml/human-activity-recognition-with-smartphones/versions/2/train.csv')
## first 5 rows
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [19]:
data['Activity'].value_counts()

Activity
LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: count, dtype: int64

In [7]:
print(f"Shape of Data is : {house_data.shape}")
print(f"Number of rows in data : {house_data.shape[0]}")
print(f"Number of columns in data : {house_data.shape[1]}")

Shape of Data is : (7352, 563)
Number of rows in data : 7352
Number of columns in data : 563


In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics.pairwise import rbf_kernel

# Load the dataset
data = pd.read_csv('/Users/ludoviclepic/.cache/kagglehub/datasets/uciml/human-activity-recognition-with-smartphones/versions/2/train.csv')

# Split features and target
X = data.iloc[:, :-1]  # All columns except the last one (features)
y = data.iloc[:, -1]   # Last column (target)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Similarité: Laplacian Score
def laplacian_score(X):
    W = rbf_kernel(X)  # Affinity matrix using RBF kernel
    D = np.diag(W.sum(axis=1))
    L = D - W  # Laplacian matrix
    scores = []
    for i in range(X.shape[1]):
        f = X[:, i]
        scores.append((f.T @ L @ f) / (f.T @ D @ f))
    return np.array(scores)

laplacian_scores = laplacian_score(X_scaled)
print("Top Laplacian Scores:", np.argsort(laplacian_scores)[:10])  # Indices of top features

### Information théorique: Mutual Information
mutual_info_scores = mutual_info_classif(X_scaled, y_encoded)
print("Top Mutual Information Scores:", np.argsort(mutual_info_scores)[::-1][:10])  # Top features

### Apprentissage parcimonieux: Lasso
lasso = Lasso(alpha=0.01, max_iter=5000)  # Alpha controls regularization strength
lasso.fit(X_scaled, y_encoded)
lasso_scores = np.abs(lasso.coef_)
print("Top Lasso Scores:", np.argsort(lasso_scores)[::-1][:10])  # Top features

### Statistiques: Chi-Square
# Min-max scaling to ensure non-negative values for Chi-Square
scaler = MinMaxScaler()
X_minmax = scaler.fit_transform(X)
chi2_scores, _ = chi2(X_minmax, y_encoded)
print("Top Chi-Square Scores:", np.argsort(chi2_scores)[::-1][:10])  # Top features

Top Laplacian Scores: [359 280  95 438 226 231 520 515 252 257]
Top Mutual Information Scores: [  9  53  89  50 216 203  90 268 310 302]
Top Lasso Scores: [268  50  58 274 381 366  52   9 317 434]
Top Chi-Square Scores: [366 367 523 287 368 234 288 510   9 549]


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the test dataset
data_test = pd.read_csv('/Users/ludoviclepic/.cache/kagglehub/datasets/uciml/human-activity-recognition-with-smartphones/versions/2/test.csv')

# Split test features and target
X_test = data_test.iloc[:, :-1]
y_test = data_test.iloc[:, -1]

# Encode the target variable for the test dataset
y_test_encoded = label_encoder.transform(y_test)

# Function to evaluate model performance on selected features
def evaluate_model(selected_features, X_train, y_train, X_test, y_test):
    # Subset the data with selected features
    X_train_subset = X_train[:, selected_features]
    X_test_subset = X_test.iloc[:, selected_features].values  # Convert to NumPy array
    
    # Train a classifier
    classifier = RandomForestClassifier(random_state=42)
    classifier.fit(X_train_subset, y_train)
    
    # Predict and evaluate accuracy
    y_pred = classifier.predict(X_test_subset)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Select top 20 features instead of 10
for x in [10, 20, 30, 50]:
    laplacian_features = np.argsort(laplacian_scores)[:x]
    mutual_info_features = np.argsort(mutual_info_scores)[::-1][:x]
    lasso_features = np.argsort(lasso_scores)[::-1][:x]
    chi2_features = np.argsort(chi2_scores)[::-1][:x]

    # Evaluate using each method's selected features
    laplacian_accuracy = evaluate_model(laplacian_features, X_scaled, y_encoded, X_test, y_test_encoded)
    mutual_info_accuracy = evaluate_model(mutual_info_features, X_scaled, y_encoded, X_test, y_test_encoded)
    lasso_accuracy = evaluate_model(lasso_features, X_scaled, y_encoded, X_test, y_test_encoded)
    chi2_accuracy = evaluate_model(chi2_features, X_scaled, y_encoded, X_test, y_test_encoded)
    print(f"Number of features: {x}")
    print(f"Accuracy using Laplacian-selected features: {laplacian_accuracy:.2f}")
    print(f"Accuracy using Mutual Information-selected features: {mutual_info_accuracy:.2f}")
    print(f"Accuracy using Lasso-selected features: {lasso_accuracy:.2f}")
    print(f"Accuracy using Chi-Square-selected features: {chi2_accuracy:.2f}")
    print("---------------------------------------------")

Number of features: 10
Accuracy using Laplacian-selected features: 0.20
Accuracy using Mutual Information-selected features: 0.19
Accuracy using Lasso-selected features: 0.31
Accuracy using Chi-Square-selected features: 0.46
---------------------------------------------
Number of features: 20
Accuracy using Laplacian-selected features: 0.20
Accuracy using Mutual Information-selected features: 0.18
Accuracy using Lasso-selected features: 0.29
Accuracy using Chi-Square-selected features: 0.32
---------------------------------------------
Number of features: 30
Accuracy using Laplacian-selected features: 0.25
Accuracy using Mutual Information-selected features: 0.18
Accuracy using Lasso-selected features: 0.26
Accuracy using Chi-Square-selected features: 0.30
---------------------------------------------
Number of features: 50
Accuracy using Laplacian-selected features: 0.24
Accuracy using Mutual Information-selected features: 0.18
Accuracy using Lasso-selected features: 0.26
Accuracy usi